In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import numpy as np
import pandas as pd
np.set_printoptions(precision=2, linewidth=120, suppress=True, edgeitems=4)
from sklearn.cross_validation import KFold

In [2]:
comm_clus = pd.read_csv('comm_clus.csv',sep=",")
comm_clus_dict = {}
i=0
while i<len(comm_clus):
    comm_clus_dict[comm_clus['comm'][i]] = comm_clus['cluster'][i]
    i+=1

In [3]:
small_set = pd.read_csv('daily_hood_type.csv')
small_set.head()

,Day,Community,Type,Arrest,Domestic,Total
0,0,11,Other,False,True,1
1,0,14,Property,False,False,1
2,0,16,Other,False,False,1
3,0,17,Property,False,False,2
4,0,22,Other,False,False,1


In [4]:
# storing the community id to community name in a dictionary
clus_dict = {}
i =0 
with open('comm_clus.csv','r') as infl:
    for line in infl.readlines():
        if i==0:
            i+=1
            continue
        [com,clus,name]=line.strip().split(',')
        com = str(int(com)-1)
        clus_dict[com]=clus
print clus_dict

{'56': '0', '77': '0', '54': '0', '42': '3', '48': '0', '43': '4', '60': '0', '61': '3', '62': '0', '63': '0', '64': '0', '49': '4', '66': '3', '67': '4', '68': '4', '69': '4', '52': '0', '53': '3', '24': '1', '25': '2', '26': '4', '27': '4', '20': '0', '21': '0', '22': '3', '23': '4', '46': '3', '47': '0', '44': '3', '45': '0', '28': '1', '29': '4', '40': '0', '41': '0', '1': '3', '3': '3', '2': '3', '5': '0', '4': '0', '7': '3', '6': '3', '9': '0', '8': '1', '51': '0', '39': '0', '65': '0', '76': '0', '75': '0', '38': '0', '73': '0', '72': '0', '71': '4', '70': '0', '59': '0', '58': '0', '11': '0', '10': '0', '13': '0', '12': '0', '15': '3', '14': '0', '17': '0', '16': '3', '19': '3', '18': '0', '31': '0', '30': '3', '37': '0', '36': '0', '35': '0', '34': '0', '33': '0', '55': '0', '74': '0', '32': '1', '57': '0', '50': '0'}


In [5]:
DoW=np.zeros(small_set.shape[0])
Month=np.zeros(small_set.shape[0])
Cluster=np.zeros(small_set.shape[0])
fd = dt.datetime.strptime('12/31/2013',"%m/%d/%Y")
for i in range(small_set.shape[0]):
    days = int(small_set['Day'][i])
    DoW[i] = (fd+dt.timedelta(days=days)).weekday()
    Month[i] = (fd+dt.timedelta(days=days)).weekday()
    com = str(int(small_set['Community'][i]))
    if com == '0':
        Cluster[i]='1'
    else:
        Cluster[i]=clus_dict[com]

In [6]:
small_set['DoW']=pd.Series(DoW)
small_set['Month']=pd.Series(Month)
small_set['Cluster']=pd.Series(Cluster)
small_set.head()

,Day,Community,Type,Arrest,Domestic,Total,DoW,Month,Cluster
0,0,11,Other,False,True,1,1.0,1.0,0.0
1,0,14,Property,False,False,1,1.0,1.0,0.0
2,0,16,Other,False,False,1,1.0,1.0,3.0
3,0,17,Property,False,False,2,1.0,1.0,0.0
4,0,22,Other,False,False,1,1.0,1.0,3.0


In [7]:
print small_set.shape

(308981, 9)


Since all the features are nominal (total is the only true number, and that is the dependent variable) we must cast the variables into dummy indexes. Luckily, Pandas can do that quickly for us.

First create new dummy dataframes and then merge the dummy frames with Total

In [8]:
dow_df=pd.get_dummies(small_set['DoW'])
month_df=pd.get_dummies(small_set['Month'])
clus_df=pd.get_dummies(small_set['Cluster'])
type_df=pd.get_dummies(small_set['Type'])

In [9]:
dow_df=dow_df.rename(index=int,columns={0:'dow_0',1:'dow_1',2:'dow_2',3:'dow_3',4:'dow_4',5:'dow_5',6:'dow_6'})
month_df=month_df.rename(index=int,columns={1:'jan',2:'feb',3:'mar',4:'apr',5:'may',6:'jun',7:'jul',8:'aug',
                                            9:'sep',10:'oct',11:'nov',12:'dec'})
clus_df=clus_df.rename(index=int,columns={0:'c0',1:'c1',2:'c2',3:'c3',4:'c4'})

In [10]:
reg_clus=pd.concat([pd.DataFrame(small_set['Total']),dow_df,month_df,clus_df,type_df],axis=1)

In [11]:
print reg_clus.head()
print reg_clus.shape

   Total  dow_0  dow_1  dow_2  dow_3  dow_4  dow_5  dow_6  0.0  jan   ...     \
0      1    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0   ...      
1      1    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0   ...      
2      1    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0   ...      
3      2    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0   ...      
4      1    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0   ...      

   may  jun   c0   c1   c2   c3   c4  Other  Property  Violent  
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0    1.0       0.0      0.0  
1  0.0  0.0  1.0  0.0  0.0  0.0  0.0    0.0       1.0      0.0  
2  0.0  0.0  0.0  0.0  0.0  1.0  0.0    1.0       0.0      0.0  
3  0.0  0.0  1.0  0.0  0.0  0.0  0.0    0.0       1.0      0.0  
4  0.0  0.0  0.0  0.0  0.0  1.0  0.0    1.0       0.0      0.0  

[5 rows x 23 columns]
(308981, 23)


Now to regress. Use LinearRegression() from scikit-learn's linear_model module

In [12]:
from sklearn import linear_model as lm

In [13]:
clus_lm = lm.LinearRegression().fit(reg_clus.drop('Total',axis=1),reg_clus['Total'])

In [14]:
clus_lm.score(reg_clus.drop('Total',axis=1),reg_clus['Total'])

0.20155852895123827

Not great. The r-squared is low enough to show this regression is effectively useless. Just as comparison, we repeat the regression using neighborhoods instead of cluster.

In [15]:
comm_df=pd.get_dummies(small_set['Community'])

In [16]:
reg_comm=pd.concat([pd.DataFrame(small_set['Total']),dow_df,month_df,comm_df,type_df],axis=1)

In [17]:
reg_comm.head()

,Total,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,0.0,jan,...,71,72,73,74,75,76,77,Other,Property,Violent
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [18]:
comm_lm = lm.LinearRegression().fit(reg_comm.drop('Total',axis=1),reg_comm['Total'])

In [19]:
comm_lm.score(reg_comm.drop('Total',axis=1),reg_comm['Total'])

0.21435838597594958

Not much better. So with 77 community levels we get about the same predictive quality as with five clusters, lending some credence to the value of the clusters.

What if we repeat with daily totals, not sperated by type? We will get daily sums for the neighborhoods, disregarding the type, or whether there was an arrest or if it was domestic

In [20]:
daily_agg = small_set.groupby(by=['Day','Community'])['Total'].sum()
daily_agg[:50]

Day  Community
0    7             3
     8             2
     11            1
     14            1
     16            1
     17            2
     22            2
     24            2
     25            2
     26            1
     27            2
     28            1
     29            3
     30            1
     32            1
     39            2
     42            1
     43            3
     44            1
     48            2
     49            1
     53            1
     54            1
     60            2
     61            1
     68            1
     69            1
     70            1
     71            2
1    1            14
     2             8
     3             9
     4             6
     5             6
     6            23
     7            10
     8            56
     9             2
     10            4
     11            4
     12            3
     13            3
     14           12
     15            9
     16           10
     17            6
     18            

In [21]:
daily_agg[daily_agg.index[1]]

2

In [22]:
i=0
with open('daily_comm_totals.csv','w') as outf:
    outf.write('Day,Community,Total\n')
    while i<len(daily_agg):
        ind = daily_agg.index[i]
        s=''
        for e in ind:
            s+=str(e)+', '
        s+=str(int(daily_agg[ind]))+'\n'
        outf.write(s)
        i+=1

In [23]:
daily_set = pd.read_csv('daily_comm_totals.csv',',')
daily_set.head()

,Day,Community,Total
0,0,7,3
1,0,8,2
2,0,11,1
3,0,14,1
4,0,16,1


In [24]:
DoW=np.zeros(daily_set.shape[0])
Month=np.zeros(daily_set.shape[0])
Cluster=np.zeros(daily_set.shape[0])
fd = dt.datetime.strptime('12/31/2013',"%m/%d/%Y")
for i in range(daily_set.shape[0]):
    days = int(daily_set['Day'][i])
    DoW[i] = (fd+dt.timedelta(days=days)).weekday()
    Month[i] = (fd+dt.timedelta(days=days)).weekday()
    com = str(int(daily_set['Community'][i]))
    if com == '0':
        Cluster[i]='1'
    else:
        Cluster[i]=clus_dict[com]

In [25]:
daily_set['DoW']=pd.Series(DoW)
daily_set['Month']=pd.Series(Month)
daily_set['Cluster']=pd.Series(Cluster)
daily_set.head()

,Day,Community,Total,DoW,Month,Cluster
0,0,7,3,1.0,1.0,3.0
1,0,8,2,1.0,1.0,1.0
2,0,11,1,1.0,1.0,0.0
3,0,14,1,1.0,1.0,0.0
4,0,16,1,1.0,1.0,3.0


In [26]:
dow_df=pd.get_dummies(daily_set['DoW'])
month_df=pd.get_dummies(daily_set['Month'])
clus_df=pd.get_dummies(daily_set['Cluster'])

In [27]:
dow_df=dow_df.rename(index=int,columns={0:'dow_0',1:'dow_1',2:'dow_2',3:'dow_3',4:'dow_4',5:'dow_5',6:'dow_6'})
month_df=month_df.rename(index=int,columns={1:'jan',2:'feb',3:'mar',4:'apr',5:'may',6:'jun',7:'jul',8:'aug',
                                            9:'sep',10:'oct',11:'nov',12:'dec'})
clus_df=clus_df.rename(index=int,columns={0:'c0',1:'c1',2:'c2',3:'c3',4:'c4'})

In [28]:
reg_clus=pd.concat([pd.DataFrame(daily_set['Total']),dow_df,month_df,clus_df],axis=1)

In [29]:
print reg_clus.head()

print reg_clus.shape

   Total  dow_0  dow_1  dow_2  dow_3  dow_4  dow_5  dow_6  0.0  jan  feb  mar  \
0      3    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0  0.0  0.0   
1      2    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0  0.0  0.0   
2      1    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0  0.0  0.0   
3      1    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0  0.0  0.0   
4      1    0.0    1.0    0.0    0.0    0.0    0.0    0.0  0.0  1.0  0.0  0.0   

   apr  may  jun   c0   c1   c2   c3   c4  
0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
1  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
(76971, 20)


In [30]:
# split data into testing and training
rand = reg_clus.sample(frac=1,replace=False)
train = rand[:(4*len(rand)/5)]
test = rand[(4*len(rand)/5):]
#print train_err, test_err

daily_lm = lm.LinearRegression().fit(train.drop('Total',axis=1),train['Total'])

In [31]:
# training score
daily_lm.score(train.drop('Total',axis=1),train['Total'])

0.7407521865366562

In [32]:
# testing score
daily_lm.score(test.drop('Total',axis=1),test['Total'])

0.74341522822393047

Much much better. The r-squared testing of 0.68 is quite good, while the 0.7 training score is only slightly better indicating only small amount of overfitting. Predicting the number of crimes per neighborhood is certainly reasonable without the use of time-series techniques.